In [ ]:
# default_exp coco

In [ ]:
# hide
from IPython.display import display

In [ ]:
# export

import logging
from abc import abstractmethod
from datetime import datetime
from dataclasses_json import dataclass_json
from dataclasses import dataclass, fields, asdict, field
from dataclasses_serialization.json import JSONSerializer
from typing import *

from cocorepr.utils import sanitize_filename

logger = logging.getLogger()

# Type helpers:
X, Y, W, H = Type[int], int, int, int

In [ ]:
# export

@dataclass_json
@dataclass
class CocoElement:
    def to_dict_skip_nulls(self):
        """ Same as `self.to_dict()` but does not add those fields
            whose values are `None`.
            WARNING! If you explicitly set a `None` value to a field
                     that has a non-`None` default value, it still
                     won't be dumped and will be deserialized wrongly.
        """
        return asdict(
            self,
            dict_factory=(
                lambda kv: {
                    k: v
                    for k, v in kv
                    if v is not None
                }
            )
        )

    @property
    def collection_name(self) -> str:
        raise NotImplementedError

In [ ]:
# hide
@dataclass
class SampleCocoElement(CocoElement):
    a: Optional[int] = None
    b: int = 3

# 'b' has default 3, so we have to preserve it
d = SampleCocoElement(a=2).to_dict_skip_nulls()
assert d == {'a': 2, 'b': 3}, d

d = SampleCocoElement().to_dict_skip_nulls()
assert d == {'b': 3}, d

# 'b' gets non-default value None, so we have to preserve it
d = SampleCocoElement(b=None).to_dict_skip_nulls()
assert d == {}, d

In [ ]:
# hide
@dataclass
class SampleCocoElementList(CocoElement):
    c: List[SampleCocoElement]

# 'b' has default 3, so we have to preserve it
d = SampleCocoElementList(c=[SampleCocoElement()]).to_dict_skip_nulls()
assert d == {'c': [{'b': 3}]}, d

In [ ]:
# export

@dataclass
class CocoInfo(CocoElement):
    year: Optional[int] = None
    version: Optional[str] = None
    description: Optional[str] = None
    contributor: Optional[str] = None
    url: Optional[str] = None
    date_created: Optional[str] = None

    @property
    def collection_name(self):
        return "info"

In [ ]:
coco_info_dict = {
 'year': 2020,
 'version': 'v1',
 'description': 'desc',
 'contributor': 'me',
 'url': 'http://url',
}

coco_info = CocoInfo.from_dict(coco_info_dict)
coco_info_dict2 = coco_info.to_dict_skip_nulls()

display(coco_info)
display(coco_info_dict2)
assert coco_info_dict2 == coco_info_dict, coco_info_dict2

assert coco_info.collection_name == 'info'

CocoInfo(year=2020, version='v1', description='desc', contributor='me', url='http://url', date_created=None)

{'year': 2020,
 'version': 'v1',
 'description': 'desc',
 'contributor': 'me',
 'url': 'http://url'}

In [ ]:
# export

@dataclass
class CocoLicense(CocoElement):
    id: int
    name: str
    url: Optional[str] = None

    @property
    def collection_name(self):
        return "licenses"

In [ ]:
coco_license_dict = {
 'id': 1,
 'name': 'Attribution-NonCommercial-NoDerivatives 4.0 International',
 'url': 'https://creativecommons.org/licenses/by-nc-nd/4.0/',
}

coco_license = CocoLicense.from_dict(coco_license_dict)
coco_license_dict2 = coco_license.to_dict_skip_nulls()
display(coco_license)
display(coco_license_dict2)

assert coco_license_dict2 == coco_license_dict, coco_license_dict2
assert coco_license.collection_name == 'licenses'

# --
# test minimal required fields
assert CocoLicense.from_dict({'id': 2, 'name': 'Apache 2.0'}) == CocoLicense(id=2, name='Apache 2.0')

CocoLicense(id=1, name='Attribution-NonCommercial-NoDerivatives 4.0 International', url='https://creativecommons.org/licenses/by-nc-nd/4.0/')

{'id': 1,
 'name': 'Attribution-NonCommercial-NoDerivatives 4.0 International',
 'url': 'https://creativecommons.org/licenses/by-nc-nd/4.0/'}

In [ ]:
# export

@dataclass
class CocoImage(CocoElement):
    id: int
    coco_url: str
    width: Optional[int] = None
    height: Optional[int] = None
    license: Optional[int] = None
    file_name: Optional[str] = None
    flickr_url: Optional[str] = None
    date_captured: Optional[str] = None

    @property
    def collection_name(self):
        return "images"

In [ ]:
coco_image_dict = {
 'id': 204800,
 'license': 1,
 'coco_url': 'https://outforz.s3.amazonaws.com/media/public/content/2021/01/10/e2e76667-f7e.jpg',
 'width': 1920,
 'height': 2560,
 'file_name': 'e2e76667-f7e.jpg',
 'date_captured': '2021-01-05 13:18:13',
}


coco_image = CocoImage.from_dict(coco_image_dict)
display(coco_image)

assert coco_image.to_dict_skip_nulls() == coco_image_dict, coco_image.to_dict_skip_nulls()
assert coco_image.collection_name == 'images'

# --
# test minimal required fields
assert CocoImage.from_dict({'id': 2, 'coco_url': 'http://image'}) == CocoImage(id=2, coco_url='http://image')

CocoImage(id=204800, coco_url='https://outforz.s3.amazonaws.com/media/public/content/2021/01/10/e2e76667-f7e.jpg', width=1920, height=2560, license=1, file_name='e2e76667-f7e.jpg', flickr_url=None, date_captured='2021-01-05 13:18:13')

In [ ]:
# export

@dataclass
class CocoAnnotation(CocoElement):
    id: int
    image_id: int

@dataclass
class CocoObjectDetectionAnnotation(CocoAnnotation):
    category_id: int
    bbox: Tuple[X, Y, W, H]
    supercategory: Optional[str] = None
    area: Optional[int] = None
    iscrowd: Optional[int] = None

In [ ]:
# export

@dataclass
class CocoCategory(CocoElement):
    id: int

    @abstractmethod
    def get_alias(self):
        raise NotImplementedError


@dataclass
class CocoObjectDetectionCategory(CocoCategory):
    name: str
    supercategory: Optional[str] = None

    def get_dir_name(self):
        name = sanitize_filename(self.name)
        return f'{name}--{self.id}'

In [ ]:
c = CocoObjectDetectionCategory(id='12345678', name='Бреф Кольорова вода Евкаліпт НОВИНКА!!! 50 г')
a = c.get_dir_name()
assert a == 'Бреф_Кольорова_вода_Евкаліпт_НОВИНКА_50_г--12345678', a

In [ ]:
# export

@dataclass
class CocoDataset(CocoElement):
    images: List[CocoImage] = field(default_factory=list)
    info: CocoInfo = CocoInfo()
    licenses: List[CocoLicense] = field(default_factory=list)

    @classmethod
    def get_non_collective_elements(cls):
        # TODO: rename to get_individual_elements()
        return ['info']

    @classmethod
    def get_collective_elements(cls):
        default_self = cls()
        non_collective = set(cls.get_non_collective_elements())
        return [f.name for f in fields(default_self) if f.name not in non_collective]

    def to_full_str(self):
        return (
            f'{self.__class__.__name__}(' + \
            ', '.join(f'{k}={len(getattr(self, k))}' for k in self.get_collective_elements()) + \
            ')'
        )

In [ ]:
non_col = CocoDataset.get_non_collective_elements()
col = CocoDataset.get_collective_elements()

assert non_col == ['info'], non_col
assert col == ['images', 'licenses'], col

In [ ]:
# export

@dataclass
class CocoObjectDetectionDataset(CocoDataset):
    annotations: List[CocoObjectDetectionAnnotation] = field(default_factory=list)
    categories: List[CocoObjectDetectionCategory] = field(default_factory=list)

In [ ]:
non_col = CocoObjectDetectionDataset.get_non_collective_elements()
col = CocoObjectDetectionDataset.get_collective_elements()

assert non_col == ['info'], non_col
assert col == ['images', 'licenses', 'annotations', 'categories'], col

In [ ]:
dataset = CocoObjectDetectionDataset(
    info=CocoInfo(year=2017, version='1.0', description='COCO 2017 Dataset', contributor='COCO Consortium', url='http://cocodataset.org', date_created='2017/09/01'),
    images=[CocoImage(id=362343, coco_url='http://image')], 
    annotations=[
      CocoObjectDetectionAnnotation(id=402717, image_id=362343, category_id=10, bbox=(196.7, 254.52, 9.89, 23.19), iscrowd=0),
    ],
    categories=[CocoObjectDetectionCategory(id=10, name="person")],
)

dataset.to_dict_skip_nulls()

{'images': [{'id': 362343, 'coco_url': 'http://image'}],
 'info': {'year': 2017,
  'version': '1.0',
  'description': 'COCO 2017 Dataset',
  'contributor': 'COCO Consortium',
  'url': 'http://cocodataset.org',
  'date_created': '2017/09/01'},
 'licenses': [],
 'annotations': [{'id': 402717,
   'image_id': 362343,
   'category_id': 10,
   'bbox': (196.7, 254.52, 9.89, 23.19),
   'iscrowd': 0}],
 'categories': [{'id': 10, 'name': 'person'}]}

In [ ]:
dataset2 = CocoObjectDetectionDataset.from_dict(dataset.to_dict())
assert dataset2 == dataset, display(dataset2, dataset)

In [ ]:
dataset.to_full_str()

'CocoObjectDetectionDataset(images=1, licenses=0, annotations=1, categories=1)'

In [ ]:
d = {'info': {'year': 2017,
  'version': '1.0',
  'description': 'COCO 2017 Dataset',
  'contributor': 'COCO Consortium',
  'url': 'http://cocodataset.org',
  'date_created': '2017/09/01'},
 'images': [
     {'id': 204800,
      'width': 1920,
      'height': 2560,
      'file_name': 'e2e76667-f7e.jpg',
      'license': 1,
      'coco_url': 'https://e2e76667-f7e.jpg',
      'date_captured': '2021-01-05 13:18:13'},
 ],
 'annotations': [{'id': 402717,
   'image_id': 362343,
   'category_id': 10,
   'bbox': (196.7, 254.52, 9.89, 23.19),
   'iscrowd': 0}],
    'categories':[]}

CocoObjectDetectionDataset.from_dict(d)

CocoObjectDetectionDataset(images=[CocoImage(id=204800, coco_url='https://e2e76667-f7e.jpg', width=1920, height=2560, license=1, file_name='e2e76667-f7e.jpg', flickr_url=None, date_captured='2021-01-05 13:18:13')], info=CocoInfo(year=2017, version='1.0', description='COCO 2017 Dataset', contributor='COCO Consortium', url='http://cocodataset.org', date_created='2017/09/01'), licenses=None, annotations=[CocoObjectDetectionAnnotation(id=402717, image_id=362343, category_id=10, bbox=(196.7, 254.52, 9.89, 23.19), supercategory=None, area=None, iscrowd=0)], categories=[])

In [ ]:
raw = {
    "id": 204800,
    "license": 1,
    "coco_url": "https://e2e76667-f7e.jpg",
    "width": 1920,
    "height": 2560,
    "file_name": "e2e76667-f7e.jpg",
    "date_captured": "2021-01-05 13:18:13"
}
display(CocoImage.from_dict(raw))
del raw["date_captured"]
display(CocoImage.from_dict(raw))
del raw["id"]
try:
    display(CocoImage.from_dict(raw))
except KeyError:
    pass
else:
    assert False, "no exception"

CocoImage(id=204800, coco_url='https://outforz.s3.amazonaws.com/media/public/content/2021/01/05/e2e76667-f7e.jpg', width=1920, height=2560, license=1, file_name='e2e76667-f7e.jpg', flickr_url=None, date_captured='2021-01-05 13:18:13')

CocoImage(id=204800, coco_url='https://outforz.s3.amazonaws.com/media/public/content/2021/01/05/e2e76667-f7e.jpg', width=1920, height=2560, license=1, file_name='e2e76667-f7e.jpg', flickr_url=None, date_captured=None)

In [ ]:
# export

MAP_COCO_TYPE_TO_DATASET_CLASS = {
    "object_detection": CocoObjectDetectionDataset,
}

def get_dataset_class(coco_kind: str):
    try:
        return MAP_COCO_TYPE_TO_DATASET_CLASS[coco_kind]
    except KeyError:
        raise ValueError(f"Not supported dataset kind: {kind}")

In [ ]:
get_dataset_class("object_detection")

__main__.CocoObjectDetectionDataset

In [ ]:
# export

def merge_datasets(d1: CocoDataset, d2: CocoDataset) -> CocoDataset:
    if d1 is None:
        return d2
    if d2 is None:
        return d1
    assert isinstance(d1, CocoDataset), (type(d1), d1)
    assert isinstance(d2, CocoDataset), (type(d1), d1)
    t1 = type(d1)
    t2 = type(d2)
    assert t1 == t2, f'Cannot merge datasets: {t1} != {t2}'

    D1 = d1.to_dict()
    D2 = d2.to_dict()
    K1 = set(D1.keys())
    K2 = set(D2.keys())
    assert K1 == K2, f'Cannot merge datasets: {K1} != {K2}'

    res = {}
    for k in K1:
        if isinstance(D1[k], list):
            v1 = {x['id']: x for x in (D1[k] or [])}
            v2 = {x['id']: x for x in (D2[k] or [])}
            v_res = {}
            for i in v1:
                if i in v2 and v1[i] != v2[i]:
                    raise ValueError(f'Invalid "{k}" of id={i}: {v1[i]} != {v2[i]}')
                v_res[i] = v1[i]
            for i in v2:
                if i in v1 and v2[i] != v1[i]:
                    raise ValueError(f'Invalid "{k}" of id={i}: {v2[i]} != {v1[i]}')
                v_res[i] = v2[i]
            res[k] = sorted(v_res.values(), key=lambda x: x['id'])
        else:
            v1 = D1[k] or {}
            v2 = D2[k] or {}
            if not v1:
                res[k] = v2
            elif not v2:
                res[k] = v1
            else:
                assert v1 == v2, f'key={k}: unexpectedly: {v1} != {v2}'
                res[k] = v1

    return t1.from_dict(res)

In [ ]:
d1 = {'info': {},
 'images': [{'id': 1, 'coco_url': 'https://image1.jpg'}],
 'annotations': [{'id': 10,
   'image_id': 1,
   'category_id': 1,
   'bbox': (4,3,2,1)}],
 'categories': [{'id': 1, 'name': 'human'}]}

d2 = {'info': {},
 'images': [{'id': 1, 'coco_url': 'https://image1.jpg'},
            {'id': 2, 'coco_url': 'https://image2.jpg'}],
 'annotations': [{'id': 10,
   'image_id': 1,
   'category_id': 1,
   'bbox': (4,3,2,1)},
 {'id': 11,
   'image_id': 2,
   'category_id': 2,
   'bbox': (1,2,3,4)}],
 'categories': [{'id': 2, 'name': 'animal'}]}

In [ ]:
d_res = merge_datasets(CocoObjectDetectionDataset.from_dict(d1),
                       CocoObjectDetectionDataset.from_dict(d2))
res = d_res.to_dict_skip_nulls()
display(res)
assert res == {'images': [{'id': 1, 'coco_url': 'https://image1.jpg'},
  {'id': 2, 'coco_url': 'https://image2.jpg'}],
 'info': {},
 'licenses': [],
 'annotations': [{'id': 10,
   'image_id': 1,
   'category_id': 1,
   'bbox': (4, 3, 2, 1)},
  {'id': 11, 'image_id': 2, 'category_id': 2, 'bbox': (1, 2, 3, 4)}],
 'categories': [{'id': 1, 'name': 'human'}, {'id': 2, 'name': 'animal'}]}, res

{'images': [{'id': 1, 'coco_url': 'https://image1.jpg'},
  {'id': 2, 'coco_url': 'https://image2.jpg'}],
 'info': {},
 'licenses': [],
 'annotations': [{'id': 10,
   'image_id': 1,
   'category_id': 1,
   'bbox': (4, 3, 2, 1)},
  {'id': 11, 'image_id': 2, 'category_id': 2, 'bbox': (1, 2, 3, 4)}],
 'categories': [{'id': 1, 'name': 'human'}, {'id': 2, 'name': 'animal'}]}

In [ ]:
d2['annotations'][0]['bbox'] = (100, 101, 102, 103)
assert d1['annotations'][0]['id'] == d2['annotations'][0]['id'] \
    and d2['annotations'][0]['bbox'] != d1['annotations'][0]['bbox']

try:
    d_res = merge_datasets(CocoObjectDetectionDataset.from_dict(d1),
                           CocoObjectDetectionDataset.from_dict(d2))
except ValueError:
    pass
else:
    assert False, 'test failed'

In [ ]:
assert merge_datasets(None, CocoObjectDetectionDataset.from_dict(d2)) == CocoObjectDetectionDataset.from_dict(d2)

In [ ]:

def extract_consistent_dataset(d: CocoDataset) -> Tuple[CocoDataset, CocoDataset]:
    """This method analyses the inter-links within the provided dataset and returns
    the dataset (subset of the original one).
    """
    imgid2img = {el['id']: el for el in d['images']}
    catid2cat = {el['id']: el for el in d['categories']}
    annotations = {}
    images = {}
    categories = {}
    # TODO: analyse also licenses
    for ann in d['annotations']:
        img = imgid2img.get(ann['image_id'])
        cat = catid2cat.get(ann['category_id'])
        if img is not None and cat is not None:
            annotations[ann['id']] = ann
            images[img['id']] = img
            categories[cat['id']] = cat
    return type(d).from_dict({
        **d.to_dict(),
        'annotations': annotations,
        'images': images,
        'categories': categories,
    })

In [ ]:
extract_consistent_dataset(
    CocoObjectDetectionDataset.from_dict({
        'annotations': [
            {'id': 1, 'image_id': 10, 'category_id': 100},
            {'id': 2, 'image_id': 20, 'category_id': 200},
        ],
        'images': [
            {'id': 10, 'coco_url': 'http://url10'},
            {'id': 30, 'coco_url': 'http://url30'},
        ],
        'categories': [
            {'id': 100, 'name': 'cat1', 'bbox': [1,2,3,4]},
            {'id': 400, 'name': 'cat4', 'bbox': [1,2,3,4]},
        ],
    })
)

KeyError: 'bbox'